### 01 Google Colab から OSS LLM への接続（Ollama）
- OPTION：VS Code から Google Colab の GPUサーバに接続
- Ollama のインストールと起動（Colab 環境特有の作業）
- LLM モデルのダウンロードと動作確認
- LangChain（ChatOllama）経由での接続

**OPTION：VS Code から Google Colab の GPUサーバに接続する手順**
1. カーネルでcolabを選択
2. New Colab Serverを選択し、GPUを選択, GPUの種類（T4など）を選択
3. !nvidia-smiでGPUが選択されているか確認

In [1]:
# !nvidia-smiでGPUが選択されているか確認
# コマンドの「!」は、python ではなく、OS のコマンドという意味（シェルエスケープ）
!nvidia-smi

Sat Feb  7 09:58:57 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### Ollama のインストールと起動（Colab 環境特有の作業）
1. Google Colab に Ollama をインストールする。
2. Ollama をバックグラウンドで起動する。
3. Ollama の動作確認。

**Ollama のインストールの補足**
- !：Notebook上で Linuxのシステムコマンドを実行するための記号
- apt-get install：Linux（ColabのOS）でソフトをインストールするコマンド
- -y：インストール中に「よろしいですか？[Y/n]」と聞かれた際、自動的に「Yes（はい）」と答えるオプション
- !curl -fsSL https://ollama.com/install.sh | sh
> Linux に Ollama をダウンロード/インストールするコマンド（公式サイトより）(https://docs.ollama.com/linux#manual-install)

In [2]:
# Ollama をインストールする。
# zstd: Ollama のインストールスクリプトがアーカイブ展開に必要とする圧縮ツール
# curl でインストールスクリプトをダウンロードし、シェルで実行する。
!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (487 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current us

**Ollama の起動の補足**
- `subprocess.Popen`を使って、Ollamaのサーバーモード（serve）を起動
- `Popen`：バックグラウンドで実行させる指示
- `stdout, stderr=subprocess.DEVNULL`：Ollama の標準出力, エラー出力先を`subprocess.DEVNULL`（ゴミ箱のような場所）に変更している。（不要な出力を表示しないようにするため。）
- `time.sleep`：Ollama サーバを起動後、サーバに接続完了するまで処理を待機させる。

In [3]:
# Ollama をバックグラウンドで起動/実行
# Ollama の標準出力/エラー出力は DEVNULL に破棄
import subprocess
import time

process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(5)  # 起動を待つ
print(f"Ollama サーバを起動しました。（PID: {process.pid}）")

Ollama サーバを起動しました。（PID: 491）


In [4]:
# Ollama の動作確認。バージョンと稼働状態を表示する。
!ollama --version
!ollama list

ollama version is 0.15.5
NAME    ID    SIZE    MODIFIED 


#### LLM モデルのダウンロードと動作確認
1. Ollama で LLM モデルをダウンロード（pull）する。
> Ollama の LLM ライブラリ（https://ollama.com/library）
    
> ライブラリにないものを使用したい場合は、Hugging Face（https://huggingface.co/） から GGUF ファイルをダウンロードし、Ollama の設定ファイル（Modelfile）を作成して、Ollama に取り込みする。本リポジトリでは、この方法は使用しない。
2. Ollama 経由で LLM に質問し、動作を確認する。

In [7]:
# Ollama で LLM モデルをダウンロードする。
# Colab だとプログレスバーが文字化けするので、出力を非表示にする。
!ollama pull gemma3:1b-it-qat > /dev/null 2>&1 && echo "Done!"

Done!


In [8]:
# ダウンロードしたモデルの一覧を確認する。
!ollama list

NAME                ID              SIZE      MODIFIED      
gemma3:1b-it-qat    b491bd3989c6    1.0 GB    2 seconds ago    


In [9]:
# モデルの詳細を確認する。
!ollama show gemma3:1b-it-qat

  Model
    architecture        gemma3     
    parameters          999.89M    
    context length      32768      
    embedding length    1152       
    quantization        Q4_0       

  Capabilities
    completion    

  Parameters
    stop           "<end_of_turn>"    
    temperature    1                  
    top_k          64                 
    top_p          0.95               



In [11]:
# Ollama CLI で直接 LLM に質問し、動作を確認する。
!ollama run gemma3:1b-it-qat "日本で一番高い山は？日本語で回答して下さい。" 2> /dev/null

日本で一番高い山は**富士山**です。



#### LangChain（ChatOllama）経由での接続
1. 必要なライブラリをインストールする。
2. ChatOllama で LLM に接続し、質問する。

In [12]:
# LangChain の Ollama 連携ライブラリをインストールする。
# -U は最新版をインストールするという指示
%pip install -U langchain-ollama

**ChatOllama の補足**
> LM Studio（ローカルLLMを扱うデスクトップアプリ）のデフォルト値を流用しています。

> ChatOllama のデフォルト値はリンク先を参照。(https://reference.langchain.com/python/integrations/langchain_ollama/?h=#langchain_ollama.ChatOllama.num_predict)

| パラメータ | 説明 | 特徴 |
| :--- | :--- | :--- |
| model | モデル名称 | Ollama の NAME と一致させる |
| num_ctx | コンテキスト長 | LLM が扱える文章（入力＋出力）の長さ |
| num_predict | 最大出力トークン数 | LLM が出力する文章の長さ（-1で無制限） |
| temperature | 回答の多様性 | 0.0 は確定的, 1.0 に近いほど回答がランダムになる |
| top_k | 候補語の制限 | LLMが出力する文章のランダムさの調整パラメータ（値が小さい:候補が絞られる） |
| top_p | 累積確率の制限 | LLMが出力する文章のランダムさの調整パラメータ（値が小さい:候補が絞られる） |
| repeat_penalty | 繰り返し抑制 | 同じ言葉のループを防ぐ。（1.0=なし, 値が大きい:抑制強い） |
| reasoning | 思考プロセスの制御 | 推論モデルで <think> タグを扱う設定（None でモデルのデフォルト設定が適用） |

In [13]:
# ChatOllama で LLM に接続し、質問する。
# ChatOllama は Ollama サーバ経由で LLM を呼び出す LangChain のラッパー。
from langchain_ollama import ChatOllama  # type: ignore
from langchain_core.messages import HumanMessage  # type: ignore

llm = ChatOllama(
    model="gemma3:1b-it-qat",
    num_ctx=4096,
    num_predict=-1,
    temperature=0.8,
    top_k=40,
    top_p=0.9,
    repeat_penalty=1.1,
    reasoning=None,
)

# LLM に質問する
messages = [HumanMessage(content="日本で一番高い山は？日本語で回答して下さい。")]
response = llm.invoke(messages)
print(response.content)

日本で一番高い山は、**富士山**です。
